In [1]:
import tensorflow as tf
import autograd
from autograd import numpy as np
#import jax
#from jax import numpy as np
import numpy as anp
import time
import timeit

import examples_lib
from examples_lib import get_logistic_ll_vec, get_logistic_ll

In [2]:
# Can't do this
#tf.constant(1, dtype=tf.float32) + tf.constant(1, dtype=tf.float64)

In [3]:

######################
# Logistic

@tf.function
def get_logistic_ll_vec(y, x_mat, theta):
    z = tf.tensordot(x_mat, theta, axes=1)
    return y * z + tf.math.log1p(z)

@tf.function
def get_logistic_ll(y, x_mat, theta):
    return tf.reduce_sum(get_logistic_ll_vec(y, x_mat, theta))


def np_get_logistic_ll_vec(y, x_mat, theta):
    z = x_mat @ theta
    return y * z + np.log1p(z)

def np_get_logistic_ll(y, x_mat, theta):
    return np.sum(np_get_logistic_ll_vec(y, x_mat, theta))


In [4]:
theta, x_mat, y = examples_lib.get_logistic_params(1000, 100)

In [5]:
np_get_logistic_ll(y, x_mat, theta)
get_logistic_ll(y, x_mat, theta)

<tf.Tensor: shape=(), dtype=float64, numpy=24.796341268369105>

In [11]:
assert np.abs(np_get_logistic_ll(y, x_mat, theta) -
              get_logistic_ll(y, x_mat, theta)) < 1e-12

assert np.max(np.abs(
    np_get_logistic_ll_vec(y, x_mat, theta) -
    get_logistic_ll_vec(y, x_mat, theta))) < 1e-12

In [7]:
np_get_logistic_ll_grad = autograd.grad(lambda theta: np_get_logistic_ll(y, x_mat, theta))
np_grad = np_get_logistic_ll_grad(theta)

theta_tf = tf.Variable(theta)
tf_y = tf.constant(y)
tf_x_mat = tf.constant(x_mat)
def tf_get_logistic_ll_grad(theta):
    with tf.GradientTape() as tape:
        ll = get_logistic_ll(tf_y, tf_x_mat, theta_tf)
    tf_grad = tape.gradient(ll, theta_tf)
    return tf_grad

tf_grad = tf_get_logistic_ll_grad(theta)
assert np.max(np.abs(np_grad - tf_grad)) < 1e-12

In [8]:
tf_time = timeit.timeit('tf_get_logistic_ll_grad(theta)', number=50, globals=globals())
np_time = timeit.timeit('np_get_logistic_ll_grad(theta)', number=50, globals=globals())
print('tf_time: ', tf_time)
print('np_time: ', np_time)

tf_time:  0.038136831019073725
np_time:  0.05711106298258528


In [9]:
np_get_logistic_ll_hessian = autograd.hessian(lambda theta: np_get_logistic_ll(y, x_mat, theta))
np_hessian = np_get_logistic_ll_hessian(theta)

def tf_get_logistic_ll_hessian(theta):
    with tf.GradientTape() as tape1:
        with tf.GradientTape() as tape2:
            ll = get_logistic_ll(tf_y, tf_x_mat, theta_tf)
        tf_grad = tape2.jacobian(ll, theta_tf)
    tf_hess = tape1.jacobian(tf_grad, theta_tf)
    return tf_hess

tf_hess = tf_get_logistic_ll_hessian(theta)
assert np.max(np.abs(np_hessian - tf_hess)) < 1e-8

In [10]:
tf_time = timeit.timeit('tf_get_logistic_ll_hessian(theta)', number=5, globals=globals())
np_time = timeit.timeit('np_get_logistic_ll_hessian(theta)', number=5, globals=globals())
print('tf_time: ', tf_time)
print('np_time: ', np_time)
print('ratio tf / np: ', tf_time / np_time)

tf_time:  4.802578848000849
np_time:  0.6298453279887326
ratio tf / np:  7.625013054136305
